In [1]:
import pandas as pd
import numpy as np
import glob as gb

In [2]:
system_flist = gb.glob('../../1. 후원금사복시스템결의서 2009-2018년/*.xlsx')
account_flist = gb.glob('../../2. 후원금 통장거래내역 2009년-2017년/*.xls')
print(f'{len(system_flist)}, {len(account_flist)}')


idx = -7
print(system_flist[idx])
print(account_flist[idx])

9, 9
../../1. 후원금사복시스템결의서 2009-2018년\2011년후원금사복시스템결의서.xlsx
../../2. 후원금 통장거래내역 2009년-2017년\입출금거래내역 결과2011년1월~12월.xls


In [3]:
# 1. preprocessing report statement xlsx
system_df = pd.read_excel(system_flist[idx])[['결의일자-번호','수입금액','지출금액']].iloc[ :-1]
system_df['결의일자'] = [ string[:-2] for string in system_df['결의일자-번호'] ]
system_df = system_df.drop('결의일자-번호', axis=1)
system_df = system_df.set_index('결의일자')
system_df.index = pd.to_datetime(system_df.index).date
system_df = system_df.sort_index()

print(system_df.shape)
system_df.tail()

(119, 2)


,수입금액,지출금액
2011-12-21,0,11860
2011-12-23,0,14000
2011-12-25,218,0
2011-12-25,0,30
2011-12-26,0,7000


In [4]:
# 2. preprocessing account xls
account_df = pd.read_excel(account_flist[idx], skiprows = 9)[['거래일자','입금금액(원)','출금금액(원)']]
account_df = account_df.set_index('거래일자').fillna(0).astype(int)
account_df.index = pd.to_datetime(account_df.index).date

print(account_df.shape)
account_df.tail()

(229, 2)


,입금금액(원),출금금액(원)
2011-12-21,0,1860
2011-12-21,0,10000
2011-12-23,0,14000
2011-12-25,218,0
2011-12-25,0,30


In [5]:
# 3. comparing total amount of income & spending
print('<<<<<<< 전표 점검을 시작합니다 >>>>>>>\n')

print('1. 수입/지출 총액 비교\n')

system_income = sum(system_df.수입금액)
account_income = sum(account_df["입금금액(원)"])
print(f'(1) 결의서전표 수입총액 : { system_income }원, 통장거래내역 입금총액 : { account_income }원')
print('>>> 수입총액은 정상입니다.\n') if system_income == account_income \
        else print(f'>>> 경고!!! 수입총액이 다릅니다! (차액 {system_income - account_income} 원) \n')

system_spending = sum(system_df.지출금액)
account_spending = sum(account_df['출금금액(원)'])
print(f'(2) 결의서전표 지출총액 : { system_spending }원, 통장거래내역 출금총액 : { account_spending }원')
print('>>> 지출총액은 정상입니다.\n') if system_spending == account_spending \
        else print(f'>>> 경고!!! 지출총액이 다릅니다! (차액 {system_spending - account_spending} 원) \n')

print('='*70, end='\n'*2)

<<<<<<< 전표 점검을 시작합니다 >>>>>>>

1. 수입/지출 총액 비교

(1) 결의서전표 수입총액 : 5871731원, 통장거래내역 입금총액 : 5871731원
>>> 수입총액은 정상입니다.

(2) 결의서전표 지출총액 : 7058810원, 통장거래내역 출금총액 : 6770030원
>>> 경고!!! 지출총액이 다릅니다! (차액 288780 원) 




In [63]:
# 4. migrating rows with same date - calcaulating total amount of each date
system_df_groupby = system_df.groupby(system_df.index)[['수입금액','지출금액']].sum()
# system_df_groupby

account_df_groupby = account_df.groupby(account_df.index)[['입금금액(원)','출금금액(원)']].sum()
# account_df_groupby

# 5. merging two df
merge_df = system_df_groupby.merge(account_df_groupby, how='outer',left_on=system_df_groupby.index, right_on=account_df_groupby.index)
merge_df = merge_df.rename(columns={'key_0':'날짜'}).set_index('날짜')
merge_df = merge_df.fillna(0).astype(int)
merge_df.head()


# 6. comparing incomes of each date
print('2. 수입결의서 날짜별 총액 비교\n')
unmatch_income = [ (idx, item1, item2) for (idx, item1, item2) 
     in zip(merge_df.index, merge_df['수입금액'], merge_df['입금금액(원)'])
     if item1 != item2 ]
unmatch_income.sort()

if not unmatch_income:
    print('>>> [수입 결의서] 정상입니다.\n')
else:
    print('>>> 경고!!! [수입 결의서] 문제가 발견되었습니다!\n')
    for idx, line in enumerate(unmatch_income):
        print(f'({idx+1}) {line[0]} 날짜 : [수입 결의서] {line[1]}원 → [통장 거래내역] {line[2]}원입니다.\n')
        pass
    pass

print('='*70, end='\n'*2)

# 7. comparing spendings of each date
print('3. 지출결의서 날짜별 총액 비교\n')
unmatch_cost = [ (idx, item1, item2) for (idx, item1, item2) 
     in zip(merge_df.index, merge_df['지출금액'], merge_df['출금금액(원)'])
     if item1 != item2 ]
unmatch_cost.sort()

if not unmatch_cost:
    print('>>> [지출 결의서] 정상입니다.\n')
else:
    print('>>> 경고!!! [지출 결의서] 문제가 발견되었습니다!\n')
    for idx, line in enumerate(unmatch_cost):
        print(f'({idx+1}) [지출 결의서] {line[0]} 날짜에 {line[1]}원으로 입력된 부분은 통장 거래내역에 {line[2]}원입니다.\n')
        pass
    pass

print('='*70, end='\n'*2)


3. 지출결의서 날짜별 총액 비교

>>> 경고!!! [지출 결의서] 문제가 발견되었습니다!

(1) [지출 결의서] 2011-01-20 날짜에 114000원으로 입력된 부분은 통장 거래내역에 70000원입니다.

(2) [지출 결의서] 2011-01-21 날짜에 0원으로 입력된 부분은 통장 거래내역에 44000원입니다.

(3) [지출 결의서] 2011-02-01 날짜에 0원으로 입력된 부분은 통장 거래내역에 99500원입니다.

(4) [지출 결의서] 2011-02-17 날짜에 42900원으로 입력된 부분은 통장 거래내역에 9000원입니다.

(5) [지출 결의서] 2011-02-18 날짜에 0원으로 입력된 부분은 통장 거래내역에 11400원입니다.

(6) [지출 결의서] 2011-02-19 날짜에 0원으로 입력된 부분은 통장 거래내역에 22500원입니다.

(7) [지출 결의서] 2011-03-02 날짜에 139000원으로 입력된 부분은 통장 거래내역에 0원입니다.

(8) [지출 결의서] 2011-03-04 날짜에 0원으로 입력된 부분은 통장 거래내역에 50000원입니다.

(9) [지출 결의서] 2011-03-05 날짜에 0원으로 입력된 부분은 통장 거래내역에 60000원입니다.

(10) [지출 결의서] 2011-03-08 날짜에 108000원으로 입력된 부분은 통장 거래내역에 60000원입니다.

(11) [지출 결의서] 2011-03-10 날짜에 0원으로 입력된 부분은 통장 거래내역에 20000원입니다.

(12) [지출 결의서] 2011-03-12 날짜에 0원으로 입력된 부분은 통장 거래내역에 18000원입니다.

(13) [지출 결의서] 2011-03-19 날짜에 232500원으로 입력된 부분은 통장 거래내역에 112500원입니다.

(14) [지출 결의서] 2011-03-20 날짜에 0원으로 입력된 부분은 통장 거래내역에 120000원입니다.

(15) [지출 결의서] 2011-03-25 날짜에 114500원으로 입력된 부분은 통장 거래

In [60]:
a = unmatch_cost[60:]
sys_tot = sum([tup[1] for tup in a])
acc_tot = sum([tup[2] for tup in a])
print(f'{sys_tot} - {acc_tot} = {sys_tot-acc_tot}')

77000 - 20000 = 57000


In [61]:
unmatch_cost[60:]

[(datetime.date(2011, 12, 17), 70000, 20000),
 (datetime.date(2011, 12, 26), 7000, 0)]